In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
from models.aa_common.performance_metrics import *

In [6]:
task = "likely_pathogenic" # pathogenic, likely_pathogenic
model_root_and_name_tuple_list = [("dbnsfp", "sift"), ("dbnsfp", "polyphen2_HVAR"), ("dbnsfp", "cadd"), ("dbnsfp", "mvp"), ("dbnsfp", "metarnn"), ("dbnsfp", "revel"),
                                  ("tape_rao", "unirep"), ("tape_rao", "protbert"), ("sequnet_dunham", "sequnet"), 
                                  ("esm_rives", "esm1b_t33_650M_UR50S"), ("esm_rives", "esm1v_t33_650M_UR90S"), ("esm_rives", "esm2_t33_650M_UR50D"),
                                  ("bioembeddings_dallago", "plus_rnn"), 
                                  ("bioembeddings_dallago", "prottrans_bert_bfd"), ("bioembeddings_dallago", "prottrans_albert_bfd"),
                                  ("bioembeddings_dallago", "prottrans_xlnet_uniref100"), 
                                  ("bioembeddings_dallago", "prottrans_t5_bfd"), ("bioembeddings_dallago", "prottrans_t5_uniref50"), ("bioembeddings_dallago", "prottrans_t5_xl_u50")]

metrics = ["AUC-ROC", "AUC-PR", "F1-max", "Th-max", "Precision", "Recall", "Accuracy", "Balanced-accuracy", "MCC"]

In [7]:
list_of_resut_df = []
for analysis_no in range(10):
    result_df = pd.read_csv(home_dir+f"models/aa_common/merged_predictions/{task}_analysis/{analysis_no}.csv", sep="\t")
    
    result_df.loc[result_df["class"]==task, "class_numeric"] = 0
    result_df.loc[result_df["class"]=="neutral", "class_numeric"] = 1
    
    list_of_resut_df.append(result_df)
    
print(list_of_resut_df[0].shape)
print(list_of_resut_df[0].columns)

(8336, 30)
Index(['id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'class', 'sift_pred',
       'polyphen2_HVAR_pred', 'cadd_pred', 'mvp_pred', 'metarnn_pred',
       'revel_pred', 'unirep_pred', 'protbert_pred', 'sequnet_pred',
       'esm1b_t33_650M_UR50S_pred', 'esm1v_t33_650M_UR90S_pred',
       'esm2_t33_650M_UR50D_pred', 'plus_rnn_pred', 'prottrans_bert_bfd_pred',
       'prottrans_albert_bfd_pred', 'prottrans_xlnet_uniref100_pred',
       'prottrans_t5_bfd_pred', 'prottrans_t5_uniref50_pred',
       'prottrans_t5_xl_u50_pred', 'class_numeric'],
      dtype='object')


In [8]:
for analysis_no, result_df in enumerate(list_of_resut_df):
    output_file = home_dir+f"models/aa_common/performance_analysis/{task}_analysis/{analysis_no}.csv"

    out = open(output_file, 'w')
    out.write("Models\\Metrics")
    for metric in metrics:
        out.write(f"\t{metric}")
    out.write("\n")

    for i, (model_root, model_name) in enumerate(model_root_and_name_tuple_list):
        print(model_name)
        model_pred_col = model_name + "_pred"
        result_df["pred"]=(result_df[model_pred_col]-result_df[model_pred_col].min())/(result_df[model_pred_col].max()-result_df[model_pred_col].min()) # scaling prediction scores between [0, 1]
        
        non_nan_result_df = result_df[~pd.isna(result_df[model_pred_col])]  # taking df for only non-NAN values
        
        auc_roc_score = get_auc_roc_score(non_nan_result_df)
        auc_pr_score, precisions, recalls, thresholds = get_auc_pr_score(non_nan_result_df)
        f1_max, th_max = get_f1max_and_th(precisions, recalls, thresholds)
        precision = get_precision_score(non_nan_result_df, th_max)
        recall = get_recall_score(non_nan_result_df, th_max)
        accuracy = get_accuracy_score(non_nan_result_df, th_max)
        balanced_accuracy = get_balanced_accuracy_score(non_nan_result_df, th_max)
        mcc = get_matthews_corrcoef(non_nan_result_df, th_max)
        
        permformance_scores = [auc_roc_score, auc_pr_score, f1_max, th_max, precision, recall, accuracy, balanced_accuracy, mcc]
        
        out.write(f"{model_name}")
        for score in permformance_scores:
            out.write(f"\t{score:.3f}")
        out.write("\n")
            
        
        # if i==5: break

    out.close()

sift
	AUC-ROC: 0.798
	AUC-PR: 0.810
	Best F1-Score: 0.755 at threshold: 0.003
	Precision score: 0.714 at threshold: 0.003
	Recall score: 0.802 at threshold: 0.003
	Accuracy score: 0.731 at threshold: 0.003
	Balanced accuracy score: 0.729 at threshold: 0.003
	MCC score: 0.463 at threshold: 0.003
polyphen2_HVAR
	AUC-ROC: 0.803
	AUC-PR: 0.605
	Best F1-Score: 0.697 at threshold: 0.000
	Precision score: 0.535 at threshold: 0.000
	Recall score: 1.000 at threshold: 0.000
	Accuracy score: 0.535 at threshold: 0.000
	Balanced accuracy score: 0.500 at threshold: 0.000
	MCC score: 0.000 at threshold: 0.000
cadd
	AUC-ROC: 0.789
	AUC-PR: 0.640
	Best F1-Score: 0.673 at threshold: 0.000
	Precision score: 0.507 at threshold: 0.000
	Recall score: 1.000 at threshold: 0.000
	Accuracy score: 0.507 at threshold: 0.000
	Balanced accuracy score: 0.500 at threshold: 0.000
	MCC score: 0.000 at threshold: 0.000
mvp
	AUC-ROC: 0.789
	AUC-PR: 0.650
	Best F1-Score: 0.670 at threshold: 0.000
	Precision score: 0.503 a